## Data Transfer using QR , screen & camera

### ***
Description of Method for Data Transfer Using QR Codes and Video
This method leverages QR codes and video encoding for transferring small to medium-sized data, such as images or text files, from one device to another. The key idea is to break the data into small chunks, convert each chunk into a QR code, and stream these QR codes as a video. The receiving device captures this video and decodes the QR codes in real-time, reassembling the original data.

Steps to Implement:
Data Preparation:

The data to be transferred (such as a file, image, or text) is first converted into a base64-encoded string. This ensures that binary data is converted into a string format that can be embedded into QR codes.
Data Chunking:

The base64 string is divided into smaller chunks. Each chunk is given a unique identifier (index) to ensure that the data can be reassembled in the correct order after decoding.
QR Code Generation:

For each chunk of data, a QR code is generated using a QR code library (like Python’s qrcode). Each QR code represents one chunk of the base64-encoded data.
The error correction level of the QR codes can be adjusted based on the reliability required during data transmission. A higher error correction level allows the data to be reconstructed even if part of the QR code is damaged or lost during transmission.
Video Encoding:

The generated QR codes are combined into a video stream, with each QR code being displayed as a frame in the video. A standard frame rate (e.g., 10-15 frames per second) is used to control the speed of the video playback.
The video is saved in a format that can be played back on the receiving device (e.g., MP4). Each frame of the video contains one QR code that encodes a chunk of the original data.
Transmission:

The video file can be transmitted to the receiving device via any medium, such as a file-sharing service, or it can be played on one device while the receiving device captures the video directly through its camera.
Data Capture and Decoding:

On the receiving device, the camera captures the video, frame by frame. A QR code detector is used to identify and decode the QR codes embedded in each frame.
The device extracts the encoded data from each decoded QR code and appends it to a list, ensuring that the chunks are stored in the correct order based on the index embedded in each chunk.
Data Reassembly:

Once all the QR codes have been decoded, the individual chunks of data are reassembled into the original base64 string using the index information.
The reassembled base64 string is decoded back into its original binary form, restoring the original file or data.
Saving the Data:

The reassembled binary data is saved on the receiving device in its original format, such as an image, text file, or any other data format.
Advantages:
No Network Dependency: The method does not rely on an internet connection or network and can work offline as long as the two devices are within visual range.
Cross-Platform Compatibility: The method can be implemented across different platforms (e.g., Android, iOS, or computers) as it relies on universally available components like cameras, QR code readers, and video players.
Security: Since the data is transferred through a visual medium, it is difficult for external systems to intercept or tamper with the transmission unless they have direct access to the video feed.
Use Cases:
Quick Data Transfer: Useful for quickly sharing small files or chunks of data between devices without relying on traditional wireless methods like Bluetooth or Wi-Fi.
Backup and Restore: This method can be used for secure, offline backups of small amounts of data that can be restored by simply scanning the video on a receiving device.
File Transfer in Restricted Networks: This method can work in environments where wireless data transfer is restricted (e.g., air-gapped systems) but visual data transfer is possible.
By combining QR codes with video streaming, this approach provides a simple and versatile mechanism for transferring data between devices without the need for complex networking protocols.

### **STEP**
   * 4-color HCCB
   * Input image into code
   * Code to video of 15 frames /sec
   * Video to QR and recreating data
   * Original data
   * Experiment with frame and diffrerent background light conditions(Best Done)
    

In [4]:
import base64
import qrcode
from PIL import Image
import math
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from typing import List, Tuple

## Creating 4-colors HCCB and check it viablitly to data transfer

In [7]:

# Define a set of colors to represent the HCCB codes
colors = {
    '00': (0, 0, 0),       # Black
    '01': (0, 0, 255),     # Red
    '10': (0, 255, 0),     # Green
    '11': (255, 255, 255)  # White
}


def file_to_binary(file_path):
    """Read the file and return its binary content."""
    with open(file_path, 'rb') as file:
        binary_data = file.read()
        return ''.join(format(byte, '08b') for byte in binary_data)

def create_hccb_code(binary_data, block_size=2, cell_size=4):
    """Convert binary data into a grid of colored blocks (HCCB code)."""
    # Pad binary data to ensure it's divisible by block_size
    if len(binary_data) % block_size != 0:
        binary_data += '0' * (block_size - len(binary_data) % block_size)
    
    # Convert binary data into chunks of 'block_size'
    binary_chunks = [binary_data[i:i + block_size] for i in range(0, len(binary_data), block_size)]
    
    # Determine grid size (make it as close to square as possible)
    grid_size = int(np.ceil(np.sqrt(len(binary_chunks))))
    hccb_image = np.zeros((grid_size * cell_size, grid_size * cell_size, 3), dtype=np.uint8)
    
    # Fill in the grid with corresponding colors
    for idx, chunk in enumerate(binary_chunks):
        row = (idx // grid_size) * cell_size
        col = (idx % grid_size) * cell_size
        
        # Get the color corresponding to the binary chunk
        color = colors.get(chunk, (0, 0, 0))  # Default to black if no match
        hccb_image[row:row + cell_size, col:col + cell_size] = color
    
    return hccb_image

def save_hccb_image(hccb_image, output_path):
    """Save the generated HCCB image to a file."""
    cv2.imwrite(output_path, hccb_image)

# Example usage
file_path = 'input.txt'  # Replace with the path to your file
output_path = 'hccb_image.png'

# Step 1: Convert file to binary data
binary_data = file_to_binary(file_path)

# Step 2: Create HCCB code (grid of colored cells)
hccb_image = create_hccb_code(binary_data)

# Step 3: Save the HCCB image
save_hccb_image(hccb_image, output_path)

print(f"HCCB code saved as {output_path}")


HCCB code saved as hccb_image.png


In [8]:
# Define the reverse color map to decode HCCB
color_to_bits = {
    (0, 0, 0): '00',        # Black
    (0, 0, 255): '01',      # Red
    (0, 255, 0): '10',      # Green
    (255, 255, 255): '11'   # White
}

def load_hccb_image(image_path):
    """Load the HCCB image as a numpy array."""
    hccb_image = cv2.imread(image_path)
    return hccb_image

def decode_hccb_code(hccb_image, cell_size=4):
    """Decode the HCCB image back into binary data."""
    grid_size = hccb_image.shape[0] // cell_size
    binary_data = ""

    # Loop over the grid and decode each color to binary
    for row in range(0, hccb_image.shape[0], cell_size):
        for col in range(0, hccb_image.shape[1], cell_size):
            # Get the color of the current cell
            color = tuple(hccb_image[row, col][:3])
            # Convert color to corresponding binary string
            binary_data += color_to_bits.get(color, '')

    return binary_data

def binary_to_file(binary_data, output_file_path):
    """Convert binary data back into a file."""
    byte_data = bytearray()
    
    # Convert binary string back to byte data (8 bits per byte)
    for i in range(0, len(binary_data), 8):
        byte_chunk = binary_data[i:i + 8]
        if len(byte_chunk) == 8:  # Only process full bytes
            byte_data.append(int(byte_chunk, 2))
    
    # Write byte data to a file
    with open(output_file_path, 'wb') as output_file:
        output_file.write(byte_data)

# Example usage
hccb_image_path = 'hccb_image.png'  # Path to the generated HCCB image
output_file_path = 'reconstructed_file.txt'  # Path to save the reconstructed file

# Step 1: Load the HCCB image
hccb_image = load_hccb_image(hccb_image_path)

# Step 2: Decode HCCB image into binary data
binary_data = decode_hccb_code(hccb_image)

# Step 3: Convert binary data back into the original file
binary_to_file(binary_data, output_file_path)

print(f"File reconstructed and saved as {output_file_path}")


File reconstructed and saved as reconstructed_file.txt


## Creating video of HCCB generated and decoding it into raw file

In [ ]:

# Define colors and reverse mapping
colors = {'00': (0, 0, 0), '01': (0, 0, 255), '10': (0, 255, 0), '11': (255, 255, 255)}
color_to_bits = {(0, 0, 0): '00', (0, 0, 255): '01', (0, 255, 0): '10', (255, 255, 255): '11'}

def file_to_binary(file_path):
    """Read file and convert to binary string with end marker."""
    with open(file_path, 'rb') as file:
        binary_data = ''.join(format(byte, '08b') for byte in file.read())
    # Add a specific end marker to denote end of actual data
    return binary_data + '11111111'  # Mark the end of binary data

def split_data(binary_data, chunk_size=2):
    """Split binary data into 2-bit chunks."""
    return [binary_data[i:i + chunk_size] for i in range(0, len(binary_data), chunk_size)]

def create_hccb_code(data_chunks, matrix_size=49, cell_size=20):
    """Create HCCB matrices from binary chunks."""
    hccbs = []
    for start in range(0, len(data_chunks), matrix_size * matrix_size):
        hccb_matrix = np.zeros((matrix_size * cell_size, matrix_size * cell_size, 3), dtype=np.uint8)
        for idx, chunk in enumerate(data_chunks[start:start + matrix_size * matrix_size]):
            row, col = idx // matrix_size, idx % matrix_size
            hccb_matrix[row * cell_size:(row + 1) * cell_size, col * cell_size:(col + 1) * cell_size] = colors.get(chunk, (0, 0, 0))
        hccbs.append(hccb_matrix)
    return hccbs

def create_hccb_video(hccbs, output_video, fps=15):
    """Create video from HCCB matrices."""
    height, width = hccbs[0].shape[:2]
    out = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    for hccb in hccbs: out.write(hccb)
    out.release()

def decode_hccb_code(hccb_frame, cell_size=4):
    """Decode binary data from HCCB frame."""
    binary_data = ""
    for row in range(0, hccb_frame.shape[0], cell_size):
        for col in range(0, hccb_frame.shape[1], cell_size):
            color = tuple(hccb_frame[row, col][:3])
            binary_data += color_to_bits.get(color, '')
    return binary_data

def binary_to_text_file(binary_data, output_file_path):
    """Convert binary data back to text file."""
    # Trim binary data at the end marker
    end_marker = binary_data.find('11111111')
    if end_marker != -1:
        binary_data = binary_data[:end_marker]

    # Convert binary string back to byte data (8 bits per byte)
    byte_data = bytearray(int(binary_data[i:i + 8], 2) for i in range(0, len(binary_data), 8))
    with open(output_file_path, 'wb') as output_file:
        output_file.write(byte_data)

def extract_hccb_from_video(video_path, output_file_path, cell_size=4):
    """Extract and decode HCCB codes from video frames."""
    cap = cv2.VideoCapture(video_path)
    total_binary_data = ""
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        total_binary_data += decode_hccb_code(frame, cell_size)
    cap.release()
    binary_to_text_file(total_binary_data, output_file_path)

# Example usage
file_path = 'input.txt'
binary_data = file_to_binary(file_path)
data_chunks = split_data(binary_data)
hccbs = create_hccb_code(data_chunks)
create_hccb_video(hccbs, 'hccb_video.mp4')
extract_hccb_from_video('hccb_video.mp4', 'reconstructed_file.txt')


In [ ]:
def binary_to_raw_file(binary_data, output_file_path):
    """Convert binary data back to raw bytes and save to a file."""
    # Ensure binary data length is a multiple of 8 (full byte chunks)
    if len(binary_data) % 8 != 0:
        print(f"Warning: Binary data length ({len(binary_data)} bits) is not a multiple of 8.")
        padding_length = 8 - (len(binary_data) % 8)
        binary_data = binary_data.ljust(len(binary_data) + padding_length, '0')  # Pad the binary data

    # Convert binary string back to byte data
    byte_data = bytearray(int(binary_data[i:i + 8], 2) for i in range(0, len(binary_data), 8))

    # Debug: Check byte data
    if byte_data:
        print(f"Successfully decoded {len(byte_data)} bytes of data.")
    else:
        print("Error: No byte data decoded.")

    # Write byte data directly to the file
    with open(output_file_path, 'wb') as output_file:
        output_file.write(byte_data)

def extract_hccb_from_video(video_path, output_file_path, cell_size=4):
    """Extract and decode HCCB codes from video frames."""
    cap = cv2.VideoCapture(video_path)
    total_binary_data = ""
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        print(f"Processing frame {frame_count}...")

        # Decode each frame
        binary_data = decode_hccb_code(frame, cell_size)

        if binary_data:
            print(f"Frame {frame_count}: Decoded {len(binary_data)} bits of data.")
        else:
            print(f"Frame {frame_count}: No data decoded.")

        total_binary_data += binary_data

    cap.release()

    # Ensure that total binary data is non-empty
    if not total_binary_data:
        print("Error: No binary data extracted from the video.")
        return

    # Write the total binary data to a raw binary file
    binary_to_raw_file(total_binary_data, output_file_path)
    print(f"File reconstructed and saved as {output_file_path}")

# Example usage
video_path = 'hccb_video.mp4'
output_file_path = 'reconstructed_file.txt'  # Save as raw binary data
extract_hccb_from_video(video_path, output_file_path)

## QR code based

In [3]:

# Function to convert PNG to base64 string
def convert_png_to_base64(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

# Function to split data into chunks
def split_data_into_chunks(data, chunk_size):
    return [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]

# Function to create QR codes from data chunks
def create_qr_codes_from_data(data, output_folder, chunk_size=1000):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    chunks = split_data_into_chunks(data, chunk_size)
    qr_codes = []
    
    for i, chunk in enumerate(chunks):
        qr = qrcode.QRCode(
            version=10,
            error_correction=qrcode.constants.ERROR_CORRECT_L,
            box_size=10,
            border=4,
        )
        qr.add_data(chunk)
        qr.make(fit=True)
        img = qr.make_image(fill_color='black', back_color='white')
        
        # Convert PIL image to numpy array for OpenCV compatibility
        img = img.convert('RGB')  # Ensure it's in RGB format
        img_np = np.array(img)
        img_gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)  # Convert to grayscale

        qr_code_path = os.path.join(output_folder, f"qr_code_{i}.png")
        cv2.imwrite(qr_code_path, img_gray)  # Save the grayscale image
        
        qr_codes.append(img_gray)  # Append grayscale QR code array
    
    return qr_codes

# Function to create a video from QR codes
def create_video_from_qr_codes(qr_codes: List[np.ndarray], output_path: str, fps: int):
    height, width = qr_codes[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height), isColor=False)  # Black-and-white (grayscale)

    for qr_code in qr_codes:
        video_writer.write(qr_code)

    video_writer.release()

# Function to decode QR codes from video
def decode_qr_from_video(video_path: str) -> str:
    decoded_data = ''
    detector = cv2.QRCodeDetector()
    
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        data, bbox, _ = detector.detectAndDecode(frame)
        if data:
            decoded_data += data
    cap.release()
    return decoded_data

# Function to save decoded base64 string as PNG
def save_base64_to_png(data: str, output_path: str):
    img_data = base64.b64decode(data)
    with open(output_path, "wb") as file:
        file.write(img_data)

# Main execution
if __name__ == "__main__":
    input_image_path = 'input_image.jpg'  # Path to your input PNG file
    output_folder = 'qr_codes'  # Folder to save QR codes
    output_video_path = 'qr_code_video1.mp4'  # Path to save the video
    output_image_path = 'output_reconstructed2.png'  # Path to save the reconstructed image
    fps = 15  # Frames per second for the video
    chunk_size = 500  # Data chunk size for each QR code

    # Step 1: Convert PNG to base64 string
    base64_data = convert_png_to_base64(input_image_path)
    
    # Step 2: Create QR codes from base64 data
    qr_codes = create_qr_codes_from_data(base64_data, output_folder, chunk_size)

    # Step 3: Create a video from the QR codes
    create_video_from_qr_codes(qr_codes, output_video_path, fps)

    # Step 4: Decode QR codes back into base64 data from the video
    decoded_base64_data = decode_qr_from_video(output_video_path)

    # Step 5: Convert decoded base64 data back to PNG
    save_base64_to_png(decoded_base64_data, output_image_path)

    print(f"Decoded image saved as {output_image_path}")


Decoded image saved as output_reconstructed2.png
